In [1]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [2]:
train_t = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.2470, 0.2435, 0.2616])
])
test_t = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.2470, 0.2435, 0.2616])
])

In [3]:
num_workers = 4
num_workers

4

In [4]:
train_ds = datasets.CIFAR10(root="./data", train=True, download=False, transform=train_t)
test_ds = datasets.CIFAR10(root="./data", train=False, download=False, transform=test_t)

In [5]:
train_loader = DataLoader(train_ds, batch_size=32, shuffle=True, num_workers=num_workers, pin_memory=True)
test_loader = DataLoader(test_ds, batch_size=32, shuffle=False, num_workers=num_workers, pin_memory=True)

In [6]:
from models.cnns.resnet import resnet18, resnet34, resnet50, resnet101, resnet152

resnet18_model = resnet18()
resnet34_model = resnet34()
resnet50_model = resnet50()
resnet101_model = resnet101()
resnet152_model = resnet152()

from torchinfo import summary

print("ResNet18\n", summary(resnet18_model))
print("\nResNet34\n", summary(resnet34_model))
print("\nResNet50\n", summary(resnet50_model))
print("\nResNet101\n", summary(resnet101_model))
print("\nResNet152\n", summary(resnet152_model))

ResNet18
Layer (type:depth-idx)                   Param #
ResNet                                   --
├─Conv2d: 1-1                            9,408
├─BatchNorm2d: 1-2                       128
├─ReLU: 1-3                              --
├─MaxPool2d: 1-4                         --
├─Sequential: 1-5                        --
│    └─BasicBlock: 2-1                   --
│    │    └─Conv2d: 3-1                  36,864
│    │    └─BatchNorm2d: 3-2             128
│    │    └─Conv2d: 3-3                  36,864
│    │    └─BatchNorm2d: 3-4             128
│    │    └─ReLU: 3-5                    --
│    └─BasicBlock: 2-2                   --
│    │    └─Conv2d: 3-6                  36,864
│    │    └─BatchNorm2d: 3-7             128
│    │    └─Conv2d: 3-8                  36,864
│    │    └─BatchNorm2d: 3-9             128
│    │    └─ReLU: 3-10                   --
├─Sequential: 1-6                        --
│    └─BasicBlock: 2-3                   --
│    │    └─Sequential: 3-11          

In [7]:
from torch import nn
from torch.optim import Adam

criterion = nn.CrossEntropyLoss()
optimizer = Adam(resnet18_model.parameters(), lr=0.001)

In [8]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [10]:
from tqdm.notebook import tqdm

num_epochs = 3

resnet18_model.to(device)
for epoch in range(num_epochs):
    resnet18_model.train()
    running_loss = 0.0
    pbar = tqdm(train_loader, desc=f"Epoch [{epoch + 1}/{num_epochs}]", leave=True)
    for images, labels in pbar:
        images = images.to(device)
        labels = labels.to(device)

        outputs = resnet18_model(images)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        pbar.set_postfix(loss=loss.item())

    avg_loss = running_loss / len(train_loader)
    print(f"Epoch [{epoch + 1}/{num_epochs}] - Avg Loss: {avg_loss:.4f}")

Epoch [1/3]:   0%|          | 0/1563 [00:00<?, ?it/s]

Epoch [1/3] - Avg Loss: 1.6151


Epoch [2/3]:   0%|          | 0/1563 [00:00<?, ?it/s]

Epoch [2/3] - Avg Loss: 1.2348


Epoch [3/3]:   0%|          | 0/1563 [00:00<?, ?it/s]

Epoch [3/3] - Avg Loss: 1.0620


In [12]:
torch.cuda.empty_cache()

In [1]:
from omegaconf import OmegaConf
from hydra import initialize, compose

with initialize(config_path="configs", version_base="1.3.2"):
    cfg = compose(config_name="config")
print(OmegaConf.to_yaml(cfg))

model:
  block: BasicBlock
  layers:
  - 3
  - 4
  - 6
  - 3
dataset:
  name: CIFAR10
  root: ./data
  mean:
  - 0.4914
  - 0.4822
  - 0.4465
  std:
  - 0.247
  - 0.2435
  - 0.2616
  num_classes: 10
  color_channels: 3
training:
  epochs: 50
  lr: 0.1
  momentum: 0.9
  weight_decay: 0.0005
  batch_size: 64
  num_workers: 4
  seed: 42
  device: cuda
logging:
  log_dir: ./runs/${now:%Y-%m-%d_%H-%M-%S}
  tensorboard: true
  use_notebook: false



E:\miniconda3\envs\toast\Lib\site-packages\hydra\_internal\defaults_list.py:251: UserWarning: In 'config': Defaults list is missing `_self_`. See https://hydra.cc/docs/1.2/upgrades/1.0_to_1.1/default_composition_order for more information
  warnings.warn(msg, UserWarning)


In [2]:
from datasets.cifar import get_cifar10_loaders
from models.cnns.resnet import resnet18
from torchinfo import summary

train_loader, val_loader = get_cifar10_loaders(
    root=cfg.dataset.root,
    batch_size=cfg.training.batch_size,
    num_workers=cfg.training.num_workers,
    mean=cfg.dataset.mean,
    std=cfg.dataset.std
)

model = resnet18(num_classes=cfg.dataset.num_classes, color_channels=cfg.dataset.color_channels)
summary(model)

Layer (type:depth-idx)                   Param #
ResNet                                   --
├─Conv2d: 1-1                            9,408
├─BatchNorm2d: 1-2                       128
├─ReLU: 1-3                              --
├─MaxPool2d: 1-4                         --
├─Sequential: 1-5                        --
│    └─BasicBlock: 2-1                   --
│    │    └─Conv2d: 3-1                  36,864
│    │    └─BatchNorm2d: 3-2             128
│    │    └─Conv2d: 3-3                  36,864
│    │    └─BatchNorm2d: 3-4             128
│    │    └─ReLU: 3-5                    --
│    └─BasicBlock: 2-2                   --
│    │    └─Conv2d: 3-6                  36,864
│    │    └─BatchNorm2d: 3-7             128
│    │    └─Conv2d: 3-8                  36,864
│    │    └─BatchNorm2d: 3-9             128
│    │    └─ReLU: 3-10                   --
├─Sequential: 1-6                        --
│    └─BasicBlock: 2-3                   --
│    │    └─Sequential: 3-11             8,448


In [3]:
import torch.nn as nn, torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(),
                      lr=cfg.training.lr,
                      momentum=cfg.training.momentum,
                      weight_decay=cfg.training.weight_decay)

In [4]:
import torch

model.train()
running_loss = 0
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
for imgs, labels in train_loader:
    imgs, labels = imgs.to(device), labels.to(device)
    preds = model(imgs)
    loss  = criterion(preds, labels)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    running_loss += loss.item()
print(f"Train Loss: {running_loss/len(train_loader):.4f}")

Train Loss: 2.0832


In [5]:
model.eval()
correct, total = 0, 0
with torch.no_grad():
    for imgs, labels in val_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        preds = model(imgs).argmax(dim=1)
        correct += (preds==labels).sum().item()
        total   += labels.size(0)
print(f"Val  Acc : {correct/total:.4f}")

Val  Acc : 0.3997
